Load packages

In [28]:
#LOAD PACKAGES
import geopandas as gpd
import matplotlib.pyplot as plt
import json
import pandas as pd
from shapely.geometry import LineString, Point
import requests
import pandas as pd
import os
import folium

import pyproj
pyproj.datadir.set_data_dir("/opt/homebrew/share/proj")

API Calls

In [29]:
LTA_KEY = 'YOUR_API_KEY'

def fetch_data(resource_url):
    all_data = []
    skip_value = 0

    while True:
        url = f"{resource_url}?$skip={skip_value}"
        headers = {
            'AccountKey': LTA_KEY,
            'accept': 'application/json'
        }
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json()

            if len(data['value']) == 0:
                break

            all_data.extend(data['value'])
            skip_value += 500
        else:
            raise Exception(f"Failed to retrieve data. Status code: {response.status_code}")
    
    return pd.DataFrame(all_data)


data_dir = "Bus_RoutesStopsServices"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

bus_routes_url = "http://datamall2.mytransport.sg/ltaodataservice/BusRoutes"
bus_routes_df = fetch_data(bus_routes_url)
bus_routes_df.to_csv(f"{data_dir}/bus_routes.csv", index=False)

bus_stops_url = "http://datamall2.mytransport.sg/ltaodataservice/BusStops"
bus_stops_df = fetch_data(bus_stops_url)
bus_stops_df.to_csv(f"{data_dir}/bus_stops.csv", index=False)

bus_services_url = "http://datamall2.mytransport.sg/ltaodataservice/BusServices"
bus_services_df = fetch_data(bus_services_url)
bus_services_df.to_csv(f"{data_dir}/bus_services.csv", index=False)

Data Cleaning (from Krystal)

In [30]:
data_dir = "Bus_RoutesStopsServices"
bus_routes = pd.read_csv(f"{data_dir}/bus_routes.csv")
bus_services = pd.read_csv(f"{data_dir}/bus_services.csv")
bus_stops = pd.read_csv(f"{data_dir}/bus_stops.csv")

In [38]:
# bus services data: filter for trunk, and direction = 1
trunk = bus_services[bus_services['Category'] == 'TRUNK']
trunk = trunk[trunk['Direction'] == 1]

#bus routes data: filter for direction = 1 and trunk buses
bus_routes = bus_routes[bus_routes['Direction'] == 1]
merged_data = pd.merge(bus_routes, trunk[['ServiceNo']], on='ServiceNo', how='inner')
merged_data

# join to get the bus stop coordinates
final_data = pd.merge(merged_data, bus_stops, on='BusStopCode', how='left')
final_data

,ServiceNo,Operator,Direction,StopSequence,BusStopCode,Distance,WD_FirstBus,WD_LastBus,SAT_FirstBus,SAT_LastBus,SUN_FirstBus,SUN_LastBus,RoadName,Description,Latitude,Longitude
0,10,SBST,1,1,75009,0.0,0500,2300,0500,2300,0500,2300,Tampines Ctrl 1,Tampines Int,1.354076,103.943391
1,10,SBST,1,2,76059,0.6,0502,2302,0502,2302,0502,2302,Tampines Ave 5,Opp Our Tampines Hub,1.352962,103.941652
2,10,SBST,1,3,76069,1.1,0504,2304,0504,2304,0503,2304,Tampines Ave 5,Blk 147,1.348753,103.942086
3,10,SBST,1,4,96289,2.3,0508,2308,0508,2309,0507,2308,Simei Ave,Changi General Hosp,1.340055,103.948381
4,10,SBST,1,5,96109,2.7,0509,2310,0509,2311,0508,2309,Simei Ave,Opp Blk 3012,1.337371,103.950673
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14818,9B,SBST,1,25,95091,9.5,0741,0817,-,-,-,-,Nicoll Dr,SAF Ferry Ter,1.387214,103.999714
14819,9B,SBST,1,26,95131,9.7,0742,0818,-,-,-,-,Airline Rd,Police Pass Off,1.385252,103.999834
14820,9B,SBST,1,27,95141,10.2,0744,0820,-,-,-,-,Airline Rd,Aft Cargo Bldg D,1.381386,103.998191
14821,9B,SBST,1,28,95061,10.6,0745,0821,-,-,-,-,Airline Rd,Airline Hse,1.377552,103.996539


Bus Routes Plotting

In [47]:
# sg map
singapore= [1.359394, 103.814301]
map_singapore = folium.Map(location = singapore, zoom_start = 12)

# grp data by service number and direction
busroutes = final_data

#to see indiv bus routes
#busroutes = final_data[final_data['ServiceNo'].isin(['163'])]

grouped_bus_routes = busroutes.groupby(['ServiceNo', 'Direction'])

for (service_no, direction), group in grouped_bus_routes:
    
    group_sorted = group.sort_values('StopSequence') #ensure busstop for each bus is in correct sequence
    coordinates = list(zip(group_sorted['Latitude'], group_sorted['Longitude'])) #get the coordinates
    bus_route_line = LineString(coordinates) #plot the line 
    
    #plot bus routes in the map
    folium.PolyLine(
        locations = coordinates,  
        weight = 2,          
        color = 'blue',     
        opacity = 0.7,      
        popup = f"Service {service_no}"  # label with bus service number
    ).add_to(map_singapore)

# add the bus stop points in the map too
for index, row in busroutes.iterrows():
    folium.CircleMarker(
        location = [row['Latitude'], row['Longitude']],
        radius = 2,
        color = 'red',
        fill = True,
        fill_color = 'red',
        popup = f"Bus Stop: {row['BusStopCode']} (Service {row['ServiceNo']})"
    ).add_to(map_singapore)

map_singapore
